# Homework 03: Classification with Logistic Regression
Using the lead scoring dataset Bank Marketing dataset, predict whether or not a customer will sign up to the platform or not. 

In [64]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split 
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression

In [71]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Data Preparation
### Missing Values
- For categorical features, replace them with 'NA'
- For numerical features, replace with with 0.0

In [72]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [73]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [74]:
# get columns with type object
categorical = df.columns[df.dtypes == 'object'].to_list()
numerical = df.columns[df.dtypes != 'object'].to_list()
# remove target variable from numerical features
numerical.remove('converted')
print("categorical: ", categorical)
print("numerical: ", numerical)

categorical:  ['lead_source', 'industry', 'employment_status', 'location']
numerical:  ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']


In [75]:
# replace NA with 'NA' for categorical features
df[categorical] = df[categorical].fillna('NA')
df[categorical]

,lead_source,industry,employment_status,location
0,paid_ads,NA,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NA,australia
4,referral,education,self_employed,europe
...,...,...,...,...
1457,referral,manufacturing,self_employed,north_america
1458,referral,technology,student,europe
1459,paid_ads,technology,student,north_america
1460,referral,NA,self_employed,north_america


In [76]:
# replace NA with 0.0 for numerical features
df[numerical] = df[numerical].fillna(0.0)
df[numerical]

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,1,79450.0,4,0.94
1,1,46992.0,1,0.80
2,5,78796.0,3,0.69
3,2,83843.0,1,0.87
4,3,85012.0,3,0.62
...,...,...,...,...
1457,1,0.0,4,0.53
1458,3,65259.0,2,0.24
1459,1,45688.0,3,0.02
1460,5,71016.0,0,0.25


In [77]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1
What is the most frequent observation (mode) for the column industry?

In [78]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [79]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

### Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [80]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [81]:
# Compute correlation matrix
corr_matrix = df[numerical].corr()

# Unstack into pairs, drop self-correlations
corr_pairs = corr_matrix.unstack().dropna()

# Drop duplicate pairs (since corr is symmetric)
corr_pairs = corr_pairs[corr_pairs.index.get_level_values(0) < corr_pairs.index.get_level_values(1)]

# Get the pair with the highest absolute correlation
highest_corr = corr_pairs.abs().idxmax()
value = corr_pairs[highest_corr]

print("Highest correlation pair:", highest_corr)
print("Correlation value:", value)


Highest correlation pair: ('annual_income', 'interaction_count')
Correlation value: 0.027036472404814396


### Train - Val - Test Split

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value y is not in your dataframe

In [37]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42) # 25% is 20% of 80% (train_val)

In [39]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

In [41]:
y_train = df_train['converted'].to_numpy()
y_val = df_val['converted'].to_numpy()
y_test = df_test['converted'].to_numpy()

In [42]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

# Question 3
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).
Which of these variables has the biggest mutual information score?

In [46]:
scores = {}
for c in categorical:
    scores[c] = round(mutual_info_score(y_train, df_train[c]), 2)

# sort dictionary by value
sorted(scores.items(), key=lambda x: x[1], reverse=True)

[('lead_source', 0.04),
 ('industry', 0.01),
 ('employment_status', 0.01),
 ('location', 0.0)]

# Question 4
Now let's train a logistic regression.

Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.

Fit the model on the training dataset.

To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
What accuracy did you get?

In [47]:
# uses one-hot encoding to encode categorical features
from sklearn.feature_extraction import DictVectorizer

In [48]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
train_dicts

[{'lead_source': 'paid_ads',
  'industry': 'retail',
  'employment_status': 'student',
  'location': 'middle_east',
  'number_of_courses_viewed': 0,
  'annual_income': 58472.0,
  'interaction_count': 5,
  'lead_score': 0.03},
 {'lead_source': 'organic_search',
  'industry': 'manufacturing',
  'employment_status': 'student',
  'location': 'middle_east',
  'number_of_courses_viewed': 3,
  'annual_income': 71738.0,
  'interaction_count': 6,
  'lead_score': 0.77},
 {'lead_source': 'paid_ads',
  'industry': 'technology',
  'employment_status': 'employed',
  'location': 'north_america',
  'number_of_courses_viewed': 3,
  'annual_income': 81973.0,
  'interaction_count': 2,
  'lead_score': 0.59},
 {'lead_source': 'NA',
  'industry': 'technology',
  'employment_status': 'employed',
  'location': 'europe',
  'number_of_courses_viewed': 1,
  'annual_income': 74956.0,
  'interaction_count': 3,
  'lead_score': 0.34},
 {'lead_source': 'organic_search',
  'industry': 'retail',
  'employment_status': 

In [49]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_train

array([[5.8472e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.1738e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [8.1973e+04, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        3.0000e+00],
       ...,
       [8.9042e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [9.0000e-01, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.0259e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00]], shape=(876, 31))

In [52]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [53]:
# fit the model
model.fit(X_train, y_train) 

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [54]:
# transform validation data with DictVectorizer
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
# use the same Vectorizer to transform the validation data - do not fit again!
X_val = dv.transform(val_dicts)

In [55]:
# predict on validation data
y_pred = model.predict_proba(X_val)[:,1]
y_pred

array([0.61192135, 0.79982686, 0.53021239, 0.47131065, 0.57066066,
       0.44226933, 0.87127624, 0.84883201, 0.83290126, 0.61497773,
       0.5496797 , 0.78153147, 0.69039781, 0.77017166, 0.52659285,
       0.91706507, 0.53170529, 0.4212287 , 0.30146247, 0.84881703,
       0.79488745, 0.73670389, 0.44527052, 0.64838439, 0.41768628,
       0.753935  , 0.90166194, 0.33902817, 0.43181264, 0.96806835,
       0.92018722, 0.37487785, 0.65230115, 0.90650053, 0.75164139,
       0.64202132, 0.82250135, 0.83375674, 0.65911596, 0.30978651,
       0.78942144, 0.35546159, 0.96517826, 0.63389015, 0.512741  ,
       0.53230421, 0.82287846, 0.7440751 , 0.73452351, 0.68955233,
       0.46964266, 0.8453931 , 0.55635192, 0.92637968, 0.65257757,
       0.61526205, 0.63816961, 0.28303763, 0.48049725, 0.57890524,
       0.35497135, 0.62174999, 0.38960519, 0.61156037, 0.85304362,
       0.75430215, 0.89186054, 0.71946497, 0.95387698, 0.89209498,
       0.75277133, 0.33849915, 0.61376279, 0.51622145, 0.64088

In [59]:
# Check Accuracy - how many of the customers predicted as converted did convert?
conversion_decision = (y_pred > 0.5)
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = conversion_decision.astype(int)
df_pred['actual'] = y_val
df_pred['correct'] = df_pred['actual'] == df_pred['prediction']
df_pred.head()

,probability,prediction,actual,correct
0,0.611921,1,0,False
1,0.799827,1,1,True
2,0.530212,1,0,False
3,0.471311,0,0,True
4,0.570661,1,0,False


In [98]:
# 70 % accuracy on the validation set
accuracy = df_pred.correct.mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.6996587030716723


# Question 5
Let's find the least useful feature using the feature elimination technique.

Train a model using the same features and parameters as in Q4 (without rounding).

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.

For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

In [ ]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [108]:
features = categorical + numerical

answer_dict = {}

for i in range(len(features)):
    small_features = [x for j, x in enumerate(features) if j != i]
    print(f"Without feature {features[i]}")
    # train model
    train_dicts = df_train[small_features].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    model.fit(X_train, y_train) 
    # predict on validation
    val_dicts = df_val[small_features].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    y_pred = model.predict_proba(X_val)[:,1]
    conversion_decision = (y_pred > 0.5)

    df_pred = pd.DataFrame()
    df_pred['probability'] = y_pred
    df_pred['prediction'] = conversion_decision.astype(int)
    df_pred['actual'] = y_val
    df_pred['correct'] = df_pred['actual'] == df_pred['prediction']
    answer_dict[features[i]] = accuracy - df_pred.correct.mean() 
    print("accuracy:", df_pred.correct.mean())

Without feature lead_source
accuracy: 0.7030716723549488
Without feature industry
accuracy: 0.6996587030716723
Without feature employment_status
accuracy: 0.6962457337883959
Without feature location
accuracy: 0.7098976109215017
Without feature number_of_courses_viewed
accuracy: 0.5563139931740614
Without feature annual_income
accuracy: 0.8532423208191127
Without feature interaction_count
accuracy: 0.5563139931740614
Without feature lead_score
accuracy: 0.7064846416382252


In [111]:
answer_dict

{'lead_source': np.float64(-0.0034129692832765013),
 'industry': np.float64(0.0),
 'employment_status': np.float64(0.0034129692832763903),
 'location': np.float64(-0.010238907849829393),
 'number_of_courses_viewed': np.float64(0.14334470989761094),
 'annual_income': np.float64(-0.15358361774744034),
 'interaction_count': np.float64(0.14334470989761094),
 'lead_score': np.float64(-0.0068259385665528916)}

# Question 6
Now let's train a regularized logistic regression.

Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].

Train models using all the features as in Q4.

Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

In [122]:
regularization = [0.01, 0.1, 1, 10, 100]
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

for C in regularization:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)   
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:,1]
    conversion_decision = (y_pred > 0.5)

    df_pred = pd.DataFrame()
    df_pred['probability'] = y_pred
    df_pred['prediction'] = conversion_decision.astype(int)
    df_pred['actual'] = y_val
    df_pred['correct'] = df_pred['actual'] == df_pred['prediction']
    print("accuracy:", df_pred.correct.mean())
    

accuracy: 0.6996587030716723
accuracy: 0.6996587030716723
accuracy: 0.6996587030716723
accuracy: 0.6996587030716723
accuracy: 0.6996587030716723


In [123]:
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
import numpy as np

results = []
for C in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_prob = model.predict_proba(X_val)[:, 1]
    y_hat = (y_prob > 0.5).astype(int)

    acc = accuracy_score(y_val, y_hat)
    auc = roc_auc_score(y_val, y_prob)
    ll  = log_loss(y_val, y_prob)
    coef_norm = np.linalg.norm(model.coef_)

    results.append((C, acc, auc, ll, coef_norm, y_hat.mean()))
    print(f"C={C:>6} acc={acc:.4f} auc={auc:.4f} logloss={ll:.4f} ||w||={coef_norm:.3f} pos_rate={y_hat.mean():.3f}")


C=  0.01 acc=0.6997 auc=0.8636 logloss=0.5468 ||w||=0.535 pos_rate=0.809
C=   0.1 acc=0.6997 auc=0.8558 logloss=0.5435 ||w||=0.583 pos_rate=0.802
C=     1 acc=0.6997 auc=0.8549 logloss=0.5433 ||w||=0.588 pos_rate=0.802
C=    10 acc=0.6997 auc=0.8547 logloss=0.5433 ||w||=0.589 pos_rate=0.802
C=   100 acc=0.6997 auc=0.8547 logloss=0.5433 ||w||=0.589 pos_rate=0.802
